In [ ]:
import pyautogui
import time
from pynput import keyboard
import threading
import pyperclip
import csv
import os
import easyocr
import cv2

# Configurações iniciais
partidas_y = [460, 518, 566, 619, 663, 719, 774, 828, 877, 929]
screenshot_regions = [
    (292, 434, 355, 46),
    (289, 491, 353, 40),
    (289, 543, 359, 42),
    (291, 597, 357, 44),
    (292, 650, 355, 42),
    (290, 703, 358, 40),
    (291, 754, 357, 42),
    (292, 809, 359, 39),
    (292, 863, 359, 40),
    (295, 912, 356, 43),
]

# Coordenadas do botão de próxima rodada
seta_x, seta_y = 662, 308

output_csv = "resultados_brasileirao.csv"
prints_dir = "prints_partidas"
os.makedirs(prints_dir, exist_ok=True)

write_header = not os.path.exists(output_csv)

reader = easyocr.Reader(['en'], gpu=False)
interromper = False

def esc_listener():
    def on_press(key):
        global interromper
        if key == keyboard.Key.esc:
            print("🛑 Interrompido manualmente.")
            interromper = True
            return False
    with keyboard.Listener(on_press=on_press) as listener:
        listener.join()

threading.Thread(target=esc_listener, daemon=True).start()

print("⌛ Esperando 3 segundos para iniciar. Pressione ESC para cancelar.")
time.sleep(3)
pyautogui.PAUSE = 1

# BLOCO 1: Navegador e Rolagem Inicial
if not interromper:
    print("🚀 Foco no navegador...")
    pyautogui.click(949, 1052)
    time.sleep(1)

    print("🖱️ Pressionando barra de rolagem...")
    pyautogui.moveTo(1908, 104)
    pyautogui.mouseDown()

    print("📜 Arrastando até a posição desejada...")
    pyautogui.moveTo(1908, 381, duration=1)
    pyautogui.mouseUp()
    print("✅ Rolagem inicial concluída.")

# CSV
with open(output_csv, mode="a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    if write_header:
        writer.writerow(["rodada", "mandante", "visitante", "placar"])

    # 🔁 Loop por cada rodada (1 a 38)
    for rodada in range(1, 39):
        if interromper:
            print("⛔ Execução interrompida.")
            break

        print(f"\n🎯 Iniciando RODADA {rodada}...")

        # Loop pelas 10 partidas da rodada
        for i, y in enumerate(partidas_y):
            if interromper:
                print("⛔ Execução interrompida.")
                break

            print(f"\n🎮 Partida {i+1}/10 da Rodada {rodada}...")

            pyautogui.click(460, y)
            time.sleep(2)


            region = screenshot_regions[i]
            img_filename = f"rodada_{rodada}_jogo_{i+1}.png"
            img_path = os.path.join(prints_dir, img_filename)
            pyautogui.screenshot(img_path, region=region)
            print(f"📸 Screenshot salva: {img_path}")

            img = cv2.imread(img_path)
            results = reader.readtext(img)
            textos = [text.strip() for (_, text, _) in results if text.strip()]

            times = []
            gols = []
            for t in textos:
                if t.isdigit():
                    gols.append(int(t))
                else:
                    times.append(t)

            if len(times) >= 2 and len(gols) >= 2:
                mandante = times[0]
                visitante = times[1]
                placar = f"{gols[0]} x {gols[1]}"
            else:
                mandante = visitante = placar = "Erro na leitura"

            print(f"✅ [{rodada}] {mandante} vs {visitante} => {placar}")
            writer.writerow([rodada, mandante, visitante, placar])

        # 👉 Clica na seta para próxima rodada (após as 10 partidas)
        if rodada < 38 and not interromper:
            print(f"➡️ Passando para a rodada {rodada + 1}...")
            pyautogui.click(seta_x, seta_y)
            time.sleep(2)

print("\n✅ Fim da execução. Dados salvos em", output_csv)


Using CPU. Note: This module is much faster with a GPU.


⌛ Esperando 3 segundos para iniciar. Pressione ESC para cancelar.
🚀 Foco no navegador...
🖱️ Pressionando barra de rolagem...
📜 Arrastando até a posição desejada...
🛑 Interrompido manualmente.
✅ Rolagem inicial concluída.
⛔ Execução interrompida.

✅ Fim da execução. Dados salvos em resultados_brasileirao.csv


Teste com uma área maior pra printar

In [1]:
import pyautogui 
import time
from pynput import keyboard
import threading
import pyperclip
import csv
import os
import easyocr
import cv2

# 🟩 Coordenadas da área pra printar
x1, y1 = 719, 347
x2, y2 = 1158, 354
x3, y3 = 1143, 528
x4, y4 = 726, 535

x = min(x1, x4)
y = min(y1, y2)
w = max(x2, x3) - x
h = max(y3, y4) - y
screenshot_region = (x, y, w, h)

# 🟡 Coordenadas Y das 10 partidas na tela
partidas_y = [460, 518, 566, 619, 663, 719, 774, 828, 877, 929]

# 👉 Coordenada do botão de próxima rodada
seta_x, seta_y = 662, 308

# 📁 Saída
output_csv = "resultados_brasileirao_2023.csv"
prints_dir = "prints_partidas_2023"
os.makedirs(prints_dir, exist_ok=True)
write_header = not os.path.exists(output_csv)

# 🔍 OCR
reader = easyocr.Reader(['en', 'pt'], gpu=False)
interromper = False

# 🔴 Interromper com ESC
def esc_listener():
    def on_press(key):
        global interromper
        if key == keyboard.Key.esc:
            print("🛑 Interrompido manualmente.")
            interromper = True
            return False
    with keyboard.Listener(on_press=on_press) as listener:
        listener.join()

threading.Thread(target=esc_listener, daemon=True).start()

print("⌛ Esperando 3 segundos para iniciar. Pressione ESC para cancelar.")
time.sleep(3)
pyautogui.PAUSE = 1

# BLOCO 1: Navegador e Rolagem Inicial
if not interromper:
    print("🚀 Foco no navegador...")
    pyautogui.click(949, 1052)
    time.sleep(1)

    print("🖱️ Pressionando barra de rolagem...")
    pyautogui.moveTo(1908, 104)
    pyautogui.mouseDown()

    print("📜 Arrastando até a posição desejada...")
    pyautogui.moveTo(1908, 381, duration=1)
    pyautogui.mouseUp()
    print("✅ Rolagem inicial concluída.")

# CSV de resultados
with open(output_csv, mode="a", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    if write_header:
        writer.writerow(["rodada", "mandante", "visitante", "placar"])

    for rodada in range(1, 39):
        if interromper:
            print("⛔ Execução interrompida.")
            break

        print(f"\n🎯 Iniciando RODADA {rodada}...")

        for i, y in enumerate(partidas_y):
            if interromper:
                print("⛔ Execução interrompida.")
                break

            print(f"\n🎮 Partida {i+1}/10 da Rodada {rodada}...")
            pyautogui.click(460, y)
            time.sleep(2)

            img_filename = f"rodada_{rodada}_jogo_{i+1}.png"
            img_path = os.path.join(prints_dir, img_filename)
            pyautogui.screenshot(img_path, region=screenshot_region)
            print(f"📸 Screenshot salva: {img_path}")

            # OCR com EasyOCR
            img = cv2.imread(img_path)
            results = reader.readtext(img)
            textos = [text.strip() for (_, text, _) in results if text.strip()]

            # Debug OCR
            print("🧾 Textos detectados:")
            for t in textos:
                print(t)

            # Lógica de extração
            import re
            placar = None
            for t in textos:
                match = re.search(r'(\d+)\s*[-x–]\s*(\d+)', t)
                if match:
                    gols1, gols2 = match.groups()
                    placar = f"{gols1} x {gols2}"
                    break

            # Filtro de nomes de times
            ruido = {'finished', 'final', 'yn', 'ft', 'gol', 'gols'}
            candidatos = [t for t in textos if t.lower() not in ruido and re.match(r'^[A-ZÁÉÍÓÚÂÊÔÃÕ][a-záéíóúâêôãõç]{2,}$', t)]

            if placar and len(candidatos) >= 2:
                mandante, visitante = candidatos[0], candidatos[1]
            else:
                mandante = visitante = placar = "Erro na leitura"

            print(f"✅ [{rodada}] {mandante} vs {visitante} => {placar}")
            writer.writerow([rodada, mandante, visitante, placar])

        # Próxima rodada
        if rodada < 38 and not interromper:
            print(f"➡️ Passando para a rodada {rodada + 1}...")
            pyautogui.click(seta_x, seta_y)
            time.sleep(2)

print("\n✅ Fim da execução. Dados salvos em", output_csv)


Using CPU. Note: This module is much faster with a GPU.


⌛ Esperando 3 segundos para iniciar. Pressione ESC para cancelar.
🚀 Foco no navegador...
🖱️ Pressionando barra de rolagem...
📜 Arrastando até a posição desejada...
✅ Rolagem inicial concluída.

🎯 Iniciando RODADA 1...

🎮 Partida 1/10 da Rodada 1...
📸 Screenshot salva: prints_partidas_2023\rodada_1_jogo_1.png
🧾 Textos detectados:
0 - 3
Finished
América-MG
Fluminense
Lelê 72'
John Kennedy 60'
G. Cano 525
✅ [1] Erro na leitura vs Erro na leitura => Erro na leitura

🎮 Partida 2/10 da Rodada 1...
📸 Screenshot salva: prints_partidas_2023\rodada_1_jogo_2.png
🧾 Textos detectados:
2-1
Eulon
Finished
Palmeiras
Cuiabá
J.López 65'
Raniele 45'+7
Endrick 5'
FULL SCREEN VIEW
✅ [1] Eulon vs Palmeiras => 2 x 1

🎮 Partida 3/10 da Rodada 1...
📸 Screenshot salva: prints_partidas_2023\rodada_1_jogo_3.png
🧾 Textos detectados:
2 - 0
Finished
Athletico
Goiás
Christian 57"
P Henrique 36'
FULL SCREEN VIEW
✅ [1] Athletico vs Goiás => 2 x 0

🎮 Partida 4/10 da Rodada 1...
📸 Screenshot salva: prints_partidas_2023\r

In [2]:
import os
import re
import csv
import cv2
import easyocr

# 📁 Pastas e arquivos
prints_dir = "prints_partidas_2023"
output_csv = "resultados_brasileirao_2023_teste.csv"
reader = easyocr.Reader(['en', 'pt'], gpu=False)

# 🟩 Regex de time mais flexível
regex_time = r'^[A-ZÁÉÍÓÚÂÊÔÃÕ][a-zA-Záéíóúâêôãõç\-]{2,}$'
ruido = {'finished', 'final', 'ft', 'gol', 'gols'}

# 📝 Abrir CSV para escrita
with open(output_csv, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["rodada", "jogo", "mandante", "visitante", "placar", "gols"])

    for filename in sorted(os.listdir(prints_dir)):
        if not filename.endswith(".png"):
            continue

        # 🧾 Nome do arquivo → rodada e jogo
        match = re.match(r"rodada_(\d+)_jogo_(\d+)\.png", filename)
        if not match:
            continue
        rodada, jogo = match.groups()

        # 🖼️ OCR
        img_path = os.path.join(prints_dir, filename)
        img = cv2.imread(img_path)
        results = reader.readtext(img)
        textos = [text.strip() for (_, text, _) in results if text.strip()]

        # 🧠 Identificar placar
        placar = None
        for t in textos:
            match = re.search(r'(\d+)\s*[-x–]\s*(\d+)', t)
            if match:
                gols1, gols2 = match.groups()
                placar = f"{gols1} x {gols2}"
                break

        # 🎯 Identificar times
        candidatos = [t for t in textos if t.lower() not in ruido and re.match(regex_time, t)]

        if placar and len(candidatos) >= 2:
            try:
                idx_placar = textos.index(next(t for t in textos if placar.replace(" x ", "-") in t or placar.replace(" x ", "–") in t or placar in t))
            except StopIteration:
                idx_placar = -1

            if 0 <= idx_placar < len(textos) - 2:
                mandante = textos[idx_placar - 1] if idx_placar - 1 >= 0 else candidatos[0]
                visitante = textos[idx_placar + 1] if idx_placar + 1 < len(textos) else candidatos[1]
            else:
                mandante, visitante = candidatos[0], candidatos[1]
        else:
            mandante = visitante = placar = "Erro"

        # 🥅 Gols (linhas que terminam com minuto tipo "52'")
        artilheiros = [t for t in textos if re.search(r"\d{2,3}['’]", t)]

        writer.writerow([rodada, jogo, mandante, visitante, placar, "; ".join(artilheiros)])
        print(f"✅ Rodada {rodada}, Jogo {jogo}: {mandante} x {visitante} ({placar})")


Using CPU. Note: This module is much faster with a GPU.


✅ Rodada 10, Jogo 1: Erro x Erro (Erro)
✅ Rodada 10, Jogo 10: Goiás x Fluminense (2 x 2)
✅ Rodada 10, Jogo 2: RccBul x Atlético-MG (1 x 1)
✅ Rodada 10, Jogo 3: Bahia x Finished (2 x 2)
✅ Rodada 10, Jogo 4: Eulon x Eulon (1 x 1)
✅ Rodada 10, Jogo 5: Fohieln x Botafogo (2 x 0)
✅ Rodada 10, Jogo 6: América-MG x Athletico (2 x 2)
✅ Rodada 10, Jogo 7: Internacional x Finished (2 x 1)
✅ Rodada 10, Jogo 8: Erro x Erro (Erro)
✅ Rodada 10, Jogo 9: Flamengo x Grêmio (3 x 0)
✅ Rodada 11, Jogo 1: Fohieln x Cruzeiro (0 x 1)
✅ Rodada 11, Jogo 10: Bragantino x Flamengo (4 x 0)
✅ Rodada 11, Jogo 2: Erro x Erro (Erro)
✅ Rodada 11, Jogo 3: Santos x Corinthians (0 x 2)
✅ Rodada 11, Jogo 4: Bahia x Palmeiras (1 x 0)
✅ Rodada 11, Jogo 5: Fluminense x Atlético-MG (1 x 1)
✅ Rodada 11, Jogo 6: GREMIO x Grêmio (3 x 1)
✅ Rodada 11, Jogo 7: Coritiba x Internacional (0 x 1)
✅ Rodada 11, Jogo 8: Euon x Cuiabá (0 x 1)
✅ Rodada 11, Jogo 9: Vasco x Goiás (0 x 1)
✅ Rodada 12, Jogo 1: Athletico x Corinthians (1 x 0)
✅ 

csv

In [12]:
import os
import csv
import cv2
import re
import easyocr

# 📁 Caminho das imagens
pasta_prints = "prints_partidas_teste"
saida_csv = "partidas_corrigidas.csv"

# 🧠 Inicia o OCR
reader = easyocr.Reader(['pt', 'en'], gpu=False)

# 🔧 Regex para detectar placar
regex_placar = re.compile(r'(\d)\s*[-x–]\s*(\d)')

# 🧼 Palavras que não são times
ruido = {'finished', 'full', 'screen', 'view', 'rccbul', 'fohieln', 'eulon', 'forall', 'ae', 'criciumaeg'}

# 📦 Inicializa lista de resultados
resultados = []

# 📂 Varre todos os arquivos
for nome_arquivo in sorted(os.listdir(pasta_prints)):
    if not nome_arquivo.endswith('.png'):
        continue

    caminho_img = os.path.join(pasta_prints, nome_arquivo)
    print(f"📸 Processando: {nome_arquivo}")

    # OCR
    img = cv2.imread(caminho_img)
    resultados_ocr = reader.readtext(img)
    textos = [t[1].strip() for t in resultados_ocr if t[1].strip()]
    textos = [t for t in textos if t.lower() not in ruido]

    # 🧾 Mostra textos
    print("🧾 Detectado:", textos)

    # 🔍 Extrai placar
    placar_match = None
    for t in textos:
        placar_match = regex_placar.search(t)
        if placar_match:
            break

    if placar_match:
        g1, g2 = placar_match.groups()
        placar = f"{g1} x {g2}"
    else:
        placar = "Erro na leitura"

    # 📌 Extrai times (os 2 primeiros candidatos com letra maiúscula e razoável)
    candidatos = [t for t in textos if re.match(r'^[A-ZÁÉÍÓÚÂÊÔÃÕ][a-záéíóúâêôãõç]{2,}', t)]
    times = candidatos[:2] if len(candidatos) >= 2 else ["Erro na leitura", "Erro na leitura"]
    mandante, visitante = times

    # 📆 Extrai número da rodada
    rodada_match = re.search(r'rodada_(\d+)_jogo_\d+\.png', nome_arquivo)
    rodada = int(rodada_match.group(1)) if rodada_match else "?"

    # 💾 Salva
    print(f"✅ [{rodada}] {mandante} vs {visitante} => {placar}")
    resultados.append([rodada, mandante, visitante, placar])

# 💽 Escreve CSV
with open(saida_csv, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["rodada", "mandante", "visitante", "placar"])
    writer.writerows(resultados)

print(f"\n✅ Fim da execução. CSV salvo em: {saida_csv}")


Using CPU. Note: This module is much faster with a GPU.


📸 Processando: rodada_10_jogo_1.png
🧾 Detectado: ['1-2', 'Criciumaec', 'Atlético-GO', 'Criciúma', "Romão 11'", "M.Trauco 90'+13", 'Newton 60', 'FULL SCREEN VIEW']
✅ [10] Criciumaec vs Atlético-GO => 1 x 2
📸 Processando: rodada_10_jogo_10.png
🧾 Detectado: ['2-1', 'Palmeiras', 'Bragantino', "Rony 55'", "M.Fernandes 49'", "Veiga 21'", 'FULL SCREEN VIEW']
✅ [10] Palmeiras vs Bragantino => 2 x 1
📸 Processando: rodada_10_jogo_2.png
🧾 Detectado: ['(Ae', '1-1', 'Botafogo', 'Athletico', "Bastos 90'+8", "Mastriani 53'", 'FULL SCREEN VIEW']
✅ [10] Botafogo vs Athletico => 1 x 1
📸 Processando: rodada_10_jogo_3.png
🧾 Detectado: ['1 - 0', '(GrEMIo', 'Fortaleza', 'Grêmio', "J.M.Lucero 42' (Pen)", 'FULL SCREEN VIEW']
✅ [10] Fortaleza vs Grêmio => 1 x 0
📸 Processando: rodada_10_jogo_4.png
🧾 Detectado: ['2 - 0', 'Juventude', 'Vasco', "J.Carlos 90'+8", "LBarbosa 52'", 'FULL SCREEN VIEW']
✅ [10] Juventude vs Vasco => 2 x 0
📸 Processando: rodada_10_jogo_5.png
🧾 Detectado: ['0 - 1', 'São Paulo', 'Cuiabá', "

csv 2

In [13]:
import os
import csv
import cv2
import re
import easyocr

# 📁 Caminho das imagens
pasta_prints = "C:\\Users\\msdof\\Downloads\\Projeto_Time_Mandante\\prints_partidas_testes_4k\\upscayl_png_realesrgan-x4plus_4x"
saida_csv = "partidas_corrigidas4k.csv"

# 🧠 Inicia o OCR
reader = easyocr.Reader(['pt', 'en'], gpu=False)

# 🔧 Regex para detectar placar
regex_placar = re.compile(r'(\d)\s*[-x–]\s*(\d)')

# 🧼 Palavras que não são times
ruido = {'finished', 'full', 'screen', 'view', 'rccbul', 'fohieln', 'eulon', 'forall', 'ae', 'criciumaeg'}

# 📦 Inicializa lista de resultados
resultados = []

# 📂 Varre todos os arquivos
for nome_arquivo in sorted(os.listdir(pasta_prints)):
    if not nome_arquivo.endswith('.png'):
        continue

    caminho_img = os.path.join(pasta_prints, nome_arquivo)
    print(f"📸 Processando: {nome_arquivo}")

    # OCR
    img = cv2.imread(caminho_img)
    resultados_ocr = reader.readtext(img)
    textos = [t[1].strip() for t in resultados_ocr if t[1].strip()]
    textos = [t for t in textos if t.lower() not in ruido]

    # 🧾 Mostra textos
    print("🧾 Detectado:", textos)

    # 🔍 Extrai placar
    placar_match = None
    for t in textos:
        placar_match = regex_placar.search(t)
        if placar_match:
            break

    if placar_match:
        g1, g2 = placar_match.groups()
        placar = f"{g1} x {g2}"
    else:
        placar = "Erro na leitura"

    # 📌 Extrai times (os 2 primeiros candidatos com letra maiúscula e razoável)
    candidatos = [t for t in textos if re.match(r'^[A-ZÁÉÍÓÚÂÊÔÃÕ][a-záéíóúâêôãõç]{2,}', t)]
    times = candidatos[:2] if len(candidatos) >= 2 else ["Erro na leitura", "Erro na leitura"]
    mandante, visitante = times

    # 📆 Extrai número da rodada
    rodada_match = re.search(r'rodada_(\d+)_jogo_\d+\.png', nome_arquivo)
    rodada = int(rodada_match.group(1)) if rodada_match else "?"

    # 💾 Salva
    print(f"✅ [{rodada}] {mandante} vs {visitante} => {placar}")
    resultados.append([rodada, mandante, visitante, placar])

# 💽 Escreve CSV
with open(saida_csv, mode="w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["rodada", "mandante", "visitante", "placar"])
    writer.writerows(resultados)

print(f"\n✅ Fim da execução. CSV salvo em: {saida_csv}")


Using CPU. Note: This module is much faster with a GPU.


📸 Processando: rodada_10_jogo_1.png
🧾 Detectado: ['1-2', 'oniciuiA E.C_', 'Atlético-GO', 'Criciuma', "G.Romso 1l'", "M-Trauco 90'+13", "Newton 60'", 'FULL SCREEN VIEW']
✅ [10] Atlético-GO vs Criciuma => 1 x 2
📸 Processando: rodada_10_jogo_10.png
🧾 Detectado: ['RodBull]', 'n)', '2 -1', 'G6IOL', 'Palmeiras', 'RB Bragantino', "Rony 55'", "M. Fernandes 49'", "R. Veiga 21'", 'FULL SCREEN VIEW']
✅ [10] RodBull] vs Palmeiras => 2 x 1
📸 Processando: rodada_10_jogo_2.png
🧾 Detectado: ['fae', '1-1', 'Botafogo', 'Athletico', "Bastos 90'+8", "G. Mastriani 53'", 'FULL SCREEN VIEW']
✅ [10] Botafogo vs Athletico => 1 x 1
📸 Processando: rodada_10_jogo_3.png
🧾 Detectado: ['[FORTALZA', '1JJ', '1-0', 'GREIVIIO', 'IIu', 'Fortaleza', 'Grêmio', "J.MI:. Lucero 42' (Pen)", 'FULL SCREEN VIEW']
✅ [10] Fortaleza vs Grêmio => 1 x 0
📸 Processando: rodada_10_jogo_4.png
🧾 Detectado: ['2 - 0', 'Juventude', 'Vasco', "J. Carlos 90'+8", "LBarbosa 52'", 'FULL SCREEN VIEW']
✅ [10] Juventude vs Vasco => 2 x 0
📸 Processando

Solucionando erros de leitura

In [ ]:
import cv2 
import pytesseract
import re
import pandas as pd
import os

# Configurar o caminho do Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Função para extrair os dados estruturados
def extrair_partida(texto):
    padrao = r'([A-Za-zÀ-ú]+)\s+(\d)'
    partidas = re.findall(padrao, texto)
    
    if len(partidas) == 2:
        mandante, visitante = partidas
        return {
            'mandante': mandante[0],
            'placar_mandante': int(mandante[1]),
            'visitante': visitante[0],
            'placar_visitante': int(visitante[1])
        }
    return None

# Caminho da pasta com os prints
pasta_imagens = "prints_partidas"

# Lista para armazenar os resultados
resultados = []

# Percorre todos os arquivos da pasta
for nome_arquivo in os.listdir(pasta_imagens):
    if nome_arquivo.lower().endswith(('.png', '.jpg', '.jpeg')):
        caminho_completo = os.path.join(pasta_imagens, nome_arquivo)
        
        # OCR
        img = cv2.imread(caminho_completo)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 3)
        text = pytesseract.image_to_string(gray, config='--oem 3 --psm 6')

        partida = extrair_partida(text)
        
        if partida:
            partida['arquivo'] = nome_arquivo  # Opcional: inclui nome do arquivo pra rastreio
            resultados.append(partida)
        else:
            print(f"⚠️ Não foi possível extrair dados de: {nome_arquivo}")

# Salva no CSV
if resultados:
    df = pd.DataFrame(resultados)
    df.to_csv('resultados_partidas.csv', index=False)
    print("✅ CSV com todas as partidas gerado com sucesso!")
    print(df)
else:
    print("❌ Nenhuma partida reconhecida nas imagens.")


⚠️ Não foi possível extrair dados de: rodada_10_jogo_1.png
Texto bruto: @ Atienco GON 1
@ Crcuma ?

----------------------------------------
⚠️ Não foi possível extrair dados de: rodada_10_jogo_10.png
Texto bruto: @ Palmeras 2
@ 28 Bragantino L

----------------------------------------
⚠️ Não foi possível extrair dados de: rodada_10_jogo_2.png
Texto bruto: © Borat t
i .

----------------------------------------
⚠️ Não foi possível extrair dados de: rodada_10_jogo_3.png
Texto bruto: % fortaleza 1
@ Gero )

----------------------------------------
⚠️ Não foi possível extrair dados de: rodada_10_jogo_4.png
Texto bruto: @ uventude
bene °

----------------------------------------
⚠️ Não foi possível extrair dados de: rodada_10_jogo_5.png
Texto bruto: WF Sho Paulo a)
@ Cumde L

----------------------------------------
⚠️ Não foi possível extrair dados de: rodada_10_jogo_7.png
Texto bruto: @ internacional .
B Commtnans )

----------------------------------------
⚠️ Não foi possível extrair da

KeyboardInterrupt: 

Ajustando erros através de upscayl

In [16]:
import cv2
import easyocr
import os

# Inicializa EasyOCR com suporte a português
reader = easyocr.Reader(['pt', 'en'], gpu=False)

# Pasta com imagens upscayl
folder = 'prints_upscayl/upscayl_png_realesrgan-x4plus_5x'

# Resultados
resultados = []

# Loop nas imagens
for filename in sorted(os.listdir(folder)):
    if filename.endswith(('.png', '.jpg', '.jpeg')):
        img_path = os.path.join(folder, filename)
        print(f"📸 Processando: {filename}")

        # Carrega imagem
        img = cv2.imread(img_path)

        # OCR direto (imagem já está com upscale!)
        results = reader.readtext(img)

        # Filtra textos úteis
        textos = [text.strip() for (_, text, _) in results if text.strip()]
        print("🧾 Detectado:", textos)

        # Identifica times e placares
        times = []
        gols = []

        for t in textos:
            if t.isdigit():
                gols.append(int(t))
            else:
                times.append(t)

        # Garante que temos info suficiente
        if len(times) >= 2 and len(gols) >= 2:
            mandante = times[0]
            visitante = times[1]
            placar = f"{gols[0]} x {gols[1]}"
        else:
            mandante = visitante = placar = "Erro na leitura"

        print(f"✅ [{filename}] {mandante} vs {visitante} => {placar}\n")
        resultados.append((filename, mandante, visitante, placar))

# Exibe resumo final
print("\n📊 Resultado Final dos Jogos:")
for nome_arquivo, m, v, p in resultados:
    print(f"{nome_arquivo}: {m} {p} {v}")


Using CPU. Note: This module is much faster with a GPU.


📸 Processando: rodada_10_jogo_1.png
🧾 Detectado: ['Atlético-G01', '1', 'Criciúma', '2']
✅ [rodada_10_jogo_1.png] Atlético-G01 vs Criciúma => 1 x 2

📸 Processando: rodada_10_jogo_10.png
🧾 Detectado: ['Palmeiras', '2', 'RB Bragantino', '1']
✅ [rodada_10_jogo_10.png] Palmeiras vs RB Bragantino => 2 x 1

📸 Processando: rodada_10_jogo_2.png
🧾 Detectado: ['Botafogo', '1', 'Athletico', '1']
✅ [rodada_10_jogo_2.png] Botafogo vs Athletico => 1 x 1

📸 Processando: rodada_10_jogo_3.png
🧾 Detectado: ['Fortaleza', '1', 'Grêmio', '0']
✅ [rodada_10_jogo_3.png] Fortaleza vs Grêmio => 1 x 0

📸 Processando: rodada_10_jogo_4.png
🧾 Detectado: ['Juventude', '2', 'Vasco']
✅ [rodada_10_jogo_4.png] Erro na leitura vs Erro na leitura => Erro na leitura

📸 Processando: rodada_10_jogo_5.png
🧾 Detectado: ['São Paulo', 'Cuiabá', '1']
✅ [rodada_10_jogo_5.png] Erro na leitura vs Erro na leitura => Erro na leitura

📸 Processando: rodada_10_jogo_6.png
🧾 Detectado: ['Cruzeiro', '2', 'Fluminense', '0']
✅ [rodada_10_jogo

In [17]:
import csv

# Caminho do arquivo de saída
csv_path = 'resultados_placares.csv'

# Exporta os resultados
with open(csv_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Imagem', 'Mandante', 'Visitante', 'Placar'])
    writer.writerows(resultados)

print(f"💾 Resultados salvos em: {csv_path}")


💾 Resultados salvos em: resultados_placares.csv


Evitar erro de leitura

In [23]:
import cv2
import easyocr

# Caminho da imagem upscalada pelo Upscayl
caminho_imagem = "upscalada.png"

# === PRÉ-PROCESSAMENTO ===
# Carrega a imagem
img = cv2.imread(caminho_imagem)

# Converte para escala de cinza
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Aplica binarização para destacar o texto
_, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

# Salva a imagem processada para debug (opcional)
cv2.imwrite("preprocessada.png", thresh)

# === OCR COM EASYOCR ===
# Inicializa o leitor com português e inglês
reader = easyocr.Reader(['pt', 'en'])

# Realiza OCR na imagem processada
resultados = reader.readtext(thresh)

# Mostra os resultados
for bbox, texto, conf in resultados:
    print(f"Texto: {texto} (Confiança: {conf:.2f})")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Texto: Vitória (Confiança: 1.00)
Texto: Palmeiras (Confiança: 1.00)
Texto: 1 (Confiança: 1.00)


In [22]:
import cv2
import easyocr

# Caminho da imagem original
caminho_imagem = "upscalada.png"

# Carrega a imagem
img = cv2.imread(caminho_imagem)

# Converte para escala de cinza
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Aplica binarização adaptativa
adaptive = cv2.adaptiveThreshold(
    gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
    cv2.THRESH_BINARY_INV, 15, 10)

# Opcional: dilatação para reforçar números finos como "0"
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
dilated = cv2.dilate(adaptive, kernel, iterations=1)

# Salva para debug
cv2.imwrite("preprocessada_dilatada.png", dilated)

# OCR
reader = easyocr.Reader(['pt', 'en'])
resultados = reader.readtext(dilated)

# Print dos resultados
for bbox, texto, conf in resultados:
    print(f"Texto: {texto} (Confiança: {conf:.2f})")


Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


Texto: Vitória (Confiança: 1.00)
Texto: Palmeiras (Confiança: 0.74)


In [27]:
import pytesseract

# Define o caminho para o executável do tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [29]:
import cv2
import pytesseract
from PIL import Image
import numpy as np

# Caminho para o executável do Tesseract no Windows
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Caminho da imagem
image_path = 'jogo_rodada_1.png'

# Carrega e pré-processa a imagem
img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 3)
thresh = cv2.adaptiveThreshold(gray, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 11, 2)

# Extração de texto
custom_config = r'--oem 3 --psm 6'
text = pytesseract.image_to_string(thresh, config=custom_config)

print("Texto detectado:")
print(text)


Texto detectado:
& Criciuma 1
© Juventude 1



In [10]:
import cv2
import pytesseract
from PIL import Image
import numpy as np

# Caminho para o executável do Tesseract no Windows
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


image_path = 'C:\\Users\\msdof\\Downloads\\Projeto_Time_Mandante\\prints_upscayl\\upscayl_png_realesrgan-x4plus_5x\\palmeiras_vitoria_upscayl_4x_ultrasharp.png'

# Carrega e pré-processa a imagem
img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 3)
thresh = cv2.adaptiveThreshold(gray, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 11, 2)

# Extração de texto
custom_config = r'--oem 3 --psm 6'
text = pytesseract.image_to_string(thresh, config=custom_config)

print("Texto detectado:")
print(text)


Texto detectado:
* ESN
41 LO)
Finished
Vitoria Palmeiras
R. Rios 20°



In [40]:
import cv2
import pytesseract
from PIL import Image
import numpy as np

# Caminho para o executável do Tesseract no Windows
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Caminho da imagem
image_path = 'rodada_1_jogo_1.png'

# Carrega e pré-processa a imagem
img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 3)
thresh = cv2.adaptiveThreshold(gray, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 11, 2)

# Extração de texto
custom_config = r'--oem 3 --psm 6'
text = pytesseract.image_to_string(thresh, config=custom_config)

print("Texto detectado:")
print(text)


Texto detectado:
@ Crciima 1
@ Juentude L



In [9]:
import cv2
import pytesseract
from PIL import Image
import numpy as np

# Caminho para o executável do Tesseract no Windows
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Caminho da imagem
image_path = 'C:\\Users\\msdof\\Downloads\\Projeto_Time_Mandante\\prints_upscayl\\upscayl_png_realesrgan-x4plus_5x\\palmeiras_vitoria_upscayl_4x_ultrasharp.png'

# Carrega e pré-processa a imagem
img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.medianBlur(gray, 3)
thresh = cv2.adaptiveThreshold(gray, 255,
                               cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 11, 2)

# Extração de texto
custom_config = r'--oem 3 --psm 6'
text = pytesseract.image_to_string(thresh, config=custom_config)

print("Texto detectado:")
print(text)


Texto detectado:
* ESN
41 LO)
Finished
Vitoria Palmeiras
R. Rios 20°



In [8]:
import pytesseract
import cv2
import re

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
image_path = 'C:\\Users\\msdof\\Downloads\\Projeto_Time_Mandante\\prints_upscayl\\upscayl_png_realesrgan-x4plus_5x\\palmeiras_vitoria_upscayl_4x_ultrasharp.png'

def preprocess_image(path):
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    gray = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
                                 cv2.THRESH_BINARY, 15, 10)
    return gray

def extrair_texto(path):
    imagem = preprocess_image(path)
    return pytesseract.image_to_string(imagem, lang='por+eng', config='--oem 3 --psm 6')

def extrair_placar(texto):
    texto = texto.replace('\n', ' ').strip()

    # 1. Detectar placar como "0-1", "0 x 1", "0–1" etc.
    placar_match = re.search(r'(\d)\s*[-x–]\s*(\d)', texto)
    if placar_match:
        gols_mandante, gols_visitante = placar_match.groups()
    else:
        # 2. Se não detectar, tentar extrair a partir de qualquer dois dígitos próximos
        numeros = re.findall(r'\d+', texto)
        if len(numeros) >= 2:
            gols_mandante, gols_visitante = numeros[0], numeros[1]
        else:
            return None, None, None, None

    # 3. Buscar nomes dos times com base na linha que contém ambos
    times = re.findall(r'\b[A-Z][a-záéíóúãõâêôç]+', texto)
    if len(times) >= 2:
        mandante, visitante = times[0], times[1]
    else:
        mandante, visitante = "Mandante?", "Visitante?"

    return mandante, visitante, int(gols_mandante), int(gols_visitante)

# 🚀 Execução
texto = extrair_texto(image_path)
print("🧾 Texto extraído:\n", texto)

mandante, visitante, g1, g2 = extrair_placar(texto)
if mandante and visitante:
    print(f"\n✅ Resultado final:\n{mandante} {g1} x {g2} {visitante}")
else:
    print("❌ Não foi possível extrair o placar corretamente.")


🧾 Texto extraído:
 O=1 ©:
UES
Finished
Vitoria Palmeiras
O R Rios 20'


✅ Resultado final:
Finished 1 x 20 Vitoria


In [44]:
import cv2
import pytesseract
import os
import re
import pandas as pd
from fuzzywuzzy import process
from PIL import Image
import numpy as np

# Caminho do Tesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Lista de times oficiais da Série A
times_oficiais = [
    "Atlético-MG", "Atlético-GO", "Athletico", "Bahia", "Botafogo", "Bragantino", "Corinthians", "Coritiba", "Cruzeiro",
    "Cuiabá", "Flamengo", "Fluminense", "Fortaleza", "Goiás", "Grêmio", "Internacional", "Juventude", "Palmeiras",
    "Santos", "São Paulo", "Vasco", "Vitória", "Criciúma"
]

# Corrigir nomes com fuzzy matching
def corrigir_nome(nome):
    nome_corrigido, score = process.extractOne(nome, times_oficiais)
    return nome_corrigido if score >= 70 else nome  # limiar ajustável

# Regex para extrair nomes e placares
def extrair_partida(texto):
    padrao = r'([A-Za-zÀ-úçÇêÊôÔãÃéÉíÍóÓúÚ]+)\s+(\d)'
    partidas = re.findall(padrao, texto)
    
    if len(partidas) == 2:
        mandante, visitante = partidas
        mandante_nome = corrigir_nome(mandante[0])
        visitante_nome = corrigir_nome(visitante[0])
        placar_mandante = int(mandante[1])
        placar_visitante = int(visitante[1])
        
        return {
            'mandante': mandante_nome,
            'placar_mandante': placar_mandante,
            'visitante': visitante_nome,
            'placar_visitante': placar_visitante,
            'placar_formatado': f"{mandante_nome} {placar_mandante} x {placar_visitante} {visitante_nome}"
        }
    return None

# Pasta com as imagens
folder_path = 'prints_upscayl/upscayl_png_realesrgan-x4plus_5x'
dados_partidas = []

# Processa cada imagem na pasta
for nome_arquivo in os.listdir(folder_path):
    if nome_arquivo.lower().endswith(('.png', '.jpg', '.jpeg')):
        caminho_imagem = os.path.join(folder_path, nome_arquivo)
        
        img = cv2.imread(caminho_imagem)
        if img is None:
            print(f"❌ Não foi possível carregar {nome_arquivo}")
            continue
        
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        gray = cv2.medianBlur(gray, 3)
        gray = cv2.adaptiveThreshold(
            gray, 255,
            cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
            cv2.THRESH_BINARY, 11, 2
        )

        config = r'--oem 3 --psm 6'
        texto = pytesseract.image_to_string(gray, config=config)
        
        partida = extrair_partida(texto)
        if partida:
            partida['arquivo'] = nome_arquivo
            dados_partidas.append(partida)
            print(f"✅ Reconhecido e corrigido em {nome_arquivo}: {partida['placar_formatado']}")
        else:
            print(f"⚠️ Texto não reconhecido corretamente em: {nome_arquivo}")
            print(f"Texto bruto:\n{texto}\n")

# Exporta para CSV
if dados_partidas:
    df = pd.DataFrame(dados_partidas)
    df.to_csv('partidas_corrigidas.csv', index=False, encoding='utf-8-sig')
    print("\n📁 CSV 'partidas_corrigidas.csv' gerado com sucesso!")
else:
    print("❌ Nenhuma partida reconhecida com sucesso.")


⚠️ Texto não reconhecido corretamente em: rodada_10_jogo_1.png
Texto bruto:
@ Atlético-GOU 1
Criciuma °


✅ Reconhecido e corrigido em rodada_10_jogo_10.png: Palmeiras 2 x 1 Bragantino
✅ Reconhecido e corrigido em rodada_10_jogo_2.png: Bahia 4 x 4 Athletico
⚠️ Texto não reconhecido corretamente em: rodada_10_jogo_3.png
Texto bruto:
Fortaleza 1
és GrémioL] 0


⚠️ Texto não reconhecido corretamente em: rodada_10_jogo_4.png
Texto bruto:
f@ Juventude 2
¥y \Vascol] 0


✅ Reconhecido e corrigido em rodada_10_jogo_5.png: São Paulo 0 x 1 Cuiabá
✅ Reconhecido e corrigido em rodada_10_jogo_6.png: Cruzeiro 2 x 0 Fluminense
✅ Reconhecido e corrigido em rodada_10_jogo_7.png: Internacional 1 x 0 Corinthians
⚠️ Texto não reconhecido corretamente em: rodada_10_jogo_8.png
Texto bruto:
| @ Vitoria n
ge Atlético-MG 2


⚠️ Texto não reconhecido corretamente em: rodada_10_jogo_9.png
Texto bruto:
= Flamengo a
© Bahia 1


⚠️ Texto não reconhecido corretamente em: rodada_11_jogo_1.png
Texto bruto:
Criciuma °
